# ML 프로젝트
- 평가지표 : RMSLE
  + 차이(실제값 - 예측값) / N
  + 차이의 제곱, 차이의 절대값, 차이 로그변환, etc
  + 공통 원리 : 평가지표의 값은 낮으면 낮을수록 좋은 모델
- 예시
  + 모델 1 : 100
  + 모델 2 : 50
  + 좋은 모델 : 모델 2
- Kaggle 주제 : 칼로리 소모량에 미치는 영향을 탐색적으로 분석해서 칼로리 소모량을 예측

# 데이터 소개
- 신체 정보 : 성별, 나이, 키, 몸무게
- 운동 특성 : 운동 시간, 심박수, 체온
- 타겟 : 칼로리 소모량

In [1]:
import matplotlib.pyplot as plt 
# 한글출력
plt.rcParams['font.family'] = 'Malgun Gothic' #  Windows 'Malgun Gothic' 
plt.rcParams['axes.unicode_minus'] = False

In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
train.head(1)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0


In [3]:
import pandas as pd

test = pd.read_csv('test.csv')
test.head(1)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8


# 데이터 가져오기

# 머신러닝의 원리
- 데이터 + 해답 ==> 머신러닝 알고리즘 ==> 규칙
- 데이터 : 신체정보 + 운동 특성
- 해답 : 칼로리 소모량
- 프로젝트 진행하면, 대부분의 데이터는 데이터만 있음 / 해답은 존재하지 않음
  + 여러분들이 해야할 것은, 해답 정의를 해야함닝의 원리
- 데이터 + 해답 ==> 머신러닝 알고리즘 ==> 규칙
- 데이터 : 신체정보 + 운동 특성
- 해답 : 칼로리 소모량
- 프로젝트 진행하면, 대부분의 데이터는 데이터만 있음 / 해답은 존재하지 않음
  + 여러분들이 해야할 것은, 해답 정의를 해야함

# 프로세스
- 데이터 수집
- 데이터 가공 & 탐색적 데이터 분석
- 데이터셋 분리
- 모델링
- 평가지표
- 최종모델 선정
- test.csv 데이터에 적용
- submission 파일로 내보내기
- kaggle에 업로드

## 데이터 가공(전처리) & 탐색적 데이터 분석

In [4]:
# 결측치 확인
train.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

# Feature Engineering
- 수치데이터를 처리하는 방식 (Min-Max 정규화, z점수 표준화)
- 범주(문자) 데이터를 처리하는 방식 (Sex : male, female)
  + One-Hot Encoding (여기에서는 이것만)
  + Ordinal Encoding (서열 척도 : 등급)
    - female : 1, male : 0
- 시도 : 강원도 ~ 제주도(서열로 나누기가 어렵다) => 강원도 사람이 제주도 사람보다 높은등급인가?
  + 인구통계-사회과학 관점 : One-Hot Encoding
  + 결제학 관점 : Ordinal Encoding

In [5]:
# Sex 컬럼을 One-Hot Encoding, Pandas Method 존재
sex_encoded = pd.get_dummies(train['Sex'], prefix='Sex')
sex_encoded

,Sex_female,Sex_male
0,False,True
1,True,False
2,True,False
3,False,True
4,True,False
...,...,...
749995,False,True
749996,True,False
749997,False,True
749998,False,True


In [6]:
train = pd.concat([train, sex_encoded], axis = 1)
train.head(1)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Sex_female,Sex_male
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,False,True


In [7]:
train = train.drop('Sex', axis = 1)
train.head(1)

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Sex_female,Sex_male
0,0,36,189.0,82.0,26.0,101.0,41.0,150.0,False,True


## Scaling 변환

In [10]:
from sklearn.preprocessing import StandardScaler
import sklearn
import numpy as np

print(sklearn.__version__)
print(np.__version__)

1.6.1
2.2.5


In [11]:
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
train[numeric_features].head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,36,189.0,82.0,26.0,101.0,41.0


In [13]:
# RMSLE 평가지표를 따라가기 위해서 로그변환
y = np.log1p(train['Calories']) # log1p = log(1+x)

# 특성 스케일링
scaler = StandardScaler()
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
X = train[numeric_features]
X_scaled = scaler.fit_transform(X)
X_scaled[0]

array([-0.3571921 ,  1.11523482,  0.49020109,  1.2663241 ,  0.58371421,
        1.23577241])

## 두 변환된 데이터 합치기

In [16]:
X_combined = pd.concat([
    pd.DataFrame(X_scaled, columns = numeric_features),
    train[['Sex_female', 'Sex_male']]
], axis=1)

X_combined.head()

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True
1,1.487943,-0.912137,-1.083172,-0.888309,-1.109436,-0.431163,True,False
2,0.631273,-1.068088,-0.797104,-1.008011,-1.215258,-0.302938,True,False
3,-1.411555,1.349162,1.062337,1.146622,1.007002,0.851095,False,True
4,-0.225397,-0.678209,-1.011655,1.146622,0.689536,0.722869,True,False


## 학습/검증 데이터 분할
- 기본원칙 : 층화추출 (비율에 따라서 데이터셋 분리)
- 아래 코드는 층화추출이 된 상태 아님 (무작위 샘플링)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
                                    # 실험 재현성
    X_combined, y, test_size = 0.2, random_state = 42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((600000, 8), (150000, 8), (600000,), (150000,))

# 모델링
- 다른 알고리즘을 적용
  + 다중 회귀(통계 알고리즘)
  + 결정트리
  + 경사하강범(GBM)
  + 결정트리 + 경사하강법 : LightGBM, XGBoost, CatBoost
  + Deep Learning

In [19]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train) # 모델 학습 끝

DecisionTreeRegressor(random_state=42)

## 모델 평가
- RMSLE 평가 함수 적용
- 실제로는 다양한 모델을 활용해서, 가장 좋은 모델 한개를 선정, 이 과정 필수
- 반복될 것 같은 이 과정을 자동으로 돌아가게 하는 코드를 만드는 것이 이번주 과제
- 지금은 dt_model을 가장 좋은 모델이라고 가정

In [22]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score # 결정계수

# 평가지표 함수 만들기
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

y_pred = dt_model.predict(X_val)

print("검증 데이터 성능:")
print(f"RMSLE: {rmsle(y_val, y_pred):.4f}")
print(f"RMSE: {mean_squared_error(y_val, y_pred):.2f}")
print(f"R2 Score: {r2_score(y_val, y_pred):.2f}")

검증 데이터 성능:
RMSLE: 0.0246
RMSE: 0.01
R2 Score: 0.99


In [24]:
X_combined.head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True


In [25]:
test.head(1)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8


# 테스트 데이터 예측

In [28]:
# 인코딩 확인
test_sex_encoded = pd.get_dummies(test['Sex'], prefix = 'Sex')

# 수치 데이터 스케일링
test_numeric = test[numeric_features]
test_scaled = scaler.transform(test_numeric)
test_scaled

# 두 개의 데이터 결합
test_combined = pd.concat([
    pd.DataFrame(test_scaled, columns = test_numeric.columns),
    test_sex_encoded
], axis = 1)

test_combined.head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,0.235887,0.179525,0.418684,-1.008011,-0.897793,-0.302938,False,True


In [30]:
test_combined.head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,0.235887,0.179525,0.418684,-1.008011,-0.897793,-0.302938,False,True


In [29]:
X_combined.head(1)

,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,-0.357192,1.115235,0.490201,1.266324,0.583714,1.235772,False,True


In [34]:
from datetime import datetime

# 테스트 데이터 예측
test_pred = dt_model.predict(test_combined)
test_pred = np.exp(test_pred) # 지수 변환하여 원래 스케일로 복원
test_pred

# 제출 파일 생성
submission = pd.DataFrame({
    'id' : test['id'],
    'Calories' : test_pred
})

submission.head()

,id,Calories
0,750000,28.0
1,750001,111.0
2,750002,88.0
3,750003,127.0
4,750004,79.0


In [35]:
# 현재 날짜와 시간을 파일명에 포함
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
submission.to_csv(f'submission_{current_time}.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from datetime import datetime

# 데이터 로딩
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 피처 엔지니어링 추가 (BMI, Intensity)
def add_features(df):
    df['BMI'] = df['Weight'] / (df['Height']/100)**2
    df['Intensity'] = df['Heart_Rate'] * df['Duration']
    return df

train = add_features(train)
test = add_features(test)

# 원-핫 인코딩 (Sex)
train = pd.get_dummies(train, columns=['Sex'])
test = pd.get_dummies(test, columns=['Sex'])

# 특성 정의
features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'BMI', 'Intensity', 'Sex_female', 'Sex_male']

# 타겟 변수 로그 변환
y = np.log1p(train['Calories'])

# 특성 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train[features])
test_scaled = scaler.transform(test[features])

# 데이터 분할
X_tr, X_val, y_tr, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 개별 모델 정의
xgb = XGBRegressor(random_state=42)
lgb = LGBMRegressor(random_state=42)
cat = CatBoostRegressor(random_state=42, verbose=0)

# 최적의 하이퍼파라미터 튜닝 (XGBoost 기준)
params = {
    'n_estimators': [100, 300],
    'max_depth': [4, 6],
    'learning_rate': [0.01, 0.05]
}

grid = GridSearchCV(xgb, params, scoring='neg_mean_squared_error', cv=3)
grid.fit(X_tr, y_tr)

best_xgb = grid.best_estimator_
print("최적 하이퍼파라미터:", grid.best_params_)

# 앙상블 모델 구성
ensemble = VotingRegressor([
    ('xgb', best_xgb),
    ('lgb', lgb),
    ('cat', cat)
])

ensemble.fit(X_tr, y_tr)

# 평가
y_pred = ensemble.predict(X_val)

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

print("앙상블 검증 RMSLE:", rmsle(y_val, y_pred))

# 최종 예측 및 제출
final_pred = np.expm1(ensemble.predict(test_scaled))

submission = pd.DataFrame({
    'id': test['id'],
    'Calories': final_pred
})

now = datetime.now().strftime("%Y%m%d_%H%M%S")
submission.to_csv(f'submission_{now}.csv', index=False)
print(f"제출파일 생성 완료: submission_{now}.csv")
